In [55]:
inputFile=r"C:\Users\jdub\Downloads\Lab6\Lab6\small_data.xlsx"
import gurobipy as grb
import pandas as pd
data=pd.read_excel(inputFile,sheetname=None,index_col=0)
prefs=data['Preferences']
shifts=data['Shifts']

In [56]:
prefs.head()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
name,,,,,,,,,,,,,,,,,,,,,
Alexis,1,1,1,1,2,1,1,0,0,0,...,0,0,0,0,0,0,0,1,1,1
Alyssa,2,0,2,2,2,2,0,0,2,1,...,2,0,2,1,2,2,2,1,2,1
Anthony,1,1,1,2,1,2,2,2,2,2,...,2,0,1,2,2,1,1,1,2,2
Brandon,2,1,1,1,2,2,2,2,1,2,...,2,0,2,1,1,2,2,2,2,1
Brianna,2,1,2,2,1,1,1,2,2,1,...,1,2,1,1,2,2,1,1,2,0


In [57]:
shifts.head()

,day,time,persons
shift_id,,,
0,2018-04-01,Morning,3
1,2018-04-01,Evening,3
2,2018-04-01,Night,1
3,2018-04-02,Morning,3
4,2018-04-02,Evening,3


In [58]:
I = prefs.index #nurses
J = shifts.index #shifts

mod=grb.Model()
#Descision vars
X={}
for i in I:
    for j in J:
        X[i,j]=mod.addVar(lb=0, ub=1, vtype=grb.GRB.BINARY, name='x[{0},{1}]'.format(i,j))

In [151]:
numShifts = len(J) #shift count
D = range(int(numShifts/3)) #days

#Non consecutive contraint
for i in I:
    for j in J:
        if j+1 < numShifts:
            mod.addConstr(X[i,j]+X[i,j+1] <= 1)
            
#Night constraint
for i in I:
    for d in D:
        j = 3*d+2
        mod.addConstr(X[i,j]+X[i,j-2] <= 1)
        if j+2 < numShifts:
            mod.addConstr(X[i,j]+X[i,j+2] <= 1)

#Shift quota constraint
for j in J:
    mod.addConstr(sum((X[i,j] for i in I)) == shifts.loc[j,'persons'])
    
#6 shift maximum constraint
W = range(int(numShifts/21)) #weeks
for i in I:
    for w in W:
        shiftSet = range(w*21,(w+1)*21)
        mod.addConstr(sum((X[i,j] for j in shiftSet)) <= 6)

#Blackout constraint
for i in I:
    for j in J:
        if prefs.loc[i,j] == 0:
            mod.addConstr(X[i,j] == 0)

In [161]:
#Sum of preference scores
P = sum((prefs.loc[i,j] for i in I for j in J if X[i,j]==1))
    
# #Shift inequality
shift_count = []
for i in I:
    shift_count.append(sum((X[i,j] for j in J)))

#Night inequality

mod.setObjective(P - 100*(max(shift_count)-min(shift_count)) , sense=grb.GRB.MAXIMIZE)

NotImplementedError: 

In [158]:
mod.setParam('OutputFlag',False)   
mod.optimize()
print('Optimal objective: {0:.2f}'.format(mod.ObjVal))

Optimal objective: 249.00


In [159]:
solutions = []
for i,x in enumerate(X):
    solutions.append((x[0], x[1], int(mod.getVars()[i].x)))
pd.DataFrame(solutions, columns = ['Name','Shift','Value']).pivot_table(index='Name',columns='Shift')

Value                            ...                              
Shift          0  1  2  3  4  5  6  7  8  9  ... 11 12 13 14 15 16 17 18 19 20
Name                                         ...                              
Alexis          0  0  1  0  0  0  1  0  0  0 ...  0  0  0  0  0  0  0  0  1  0
Alyssa          0  0  0  1  0  0  0  0  0  0 ...  1  0  0  1  0  0  0  0  1  0
Anthony         1  0  0  0  1  0  0  0  0  0 ...  0  0  1  0  1  0  0  0  0  1
Brandon         0  1  0  0  1  0  1  0  0  1 ...  0  0  0  0  1  0  0  0  1  0
Brianna         1  0  0  0  0  0  0  1  0  1 ...  0  0  1  0  0  1  0  1  0  0
Caleb           0  0  0  1  0  0  0  1  0  0 ...  0  1  0  0  0  1  0  1  0  0
Cameron         0  1  0  1  0  0  0  1  0  0 ...  0  1  0  0  1  0  0  0  0  0
Chloe           1  0  0  0  1  0  1  0  0  1 ...  0  0  1  0  0  0  1  0  0  0
Christopher     0  1  0  0  0  1  0  0  1  0 ...  0  1  0  0  0  1  0  1  0  0

[9 rows x 21 columns]